In [1]:
%pip install -e ..
%load_ext autoreload
%autoreload 2


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Obtaining file:///C:/Users/USER/Desktop/projects/Credit%20Score%20Modelling
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for credit-score-modelling (pyproject.toml): started
  Building editable for credit-score-modelling (pyproject.toml): finished with status 'done'
  Created wheel for credit-score-modelling: filename=credit_score_modelling-0.1.0-0.editable-py3-none-any.whl size=2389 sha256=1a3e542f251e6988e4ecf31fce99cd1aa07cae3b68af98b615e6bb1abbb3ed39
  Stored in directory: C:\Users\USER\AppData\Local\Temp\pip-ephem-whee

In [12]:
import pandas as pd
from credit_score.features.load_data import load_split
from credit_score.features.cleaning import fit_cleaning_metadata, clean_and_prepare

In [13]:
# Load raw split datasets
df_train = load_split("train")
df_val   = load_split("val")
df_test  = load_split("test")

# Fit cleaning metadata on train
metadata = fit_cleaning_metadata(df_train)

In [14]:
# Clean all datasets using train-based logic
df_train = clean_and_prepare(df_train, metadata)
df_val   = clean_and_prepare(df_val, metadata)
df_test  = clean_and_prepare(df_test, metadata)

Outliers removed: 11
Outliers removed: 3
Outliers removed: 1
